In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from datetime import datetime

import pandas as pd
import time
import sys, os
import re
import json


def find_css(css_selector, browser):
    return browser.find_element(By.CSS_SELECTOR, css_selector)
def finds_css(css_selector, browser):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def find_xpath(xpath, browser):
    return browser.find_element(By.XPATH, xpath)
def finds_xpath(xpath, browser):
    return browser.find_elements(By.XPATH, xpath)

def find_id(e_id, browser):
    return browser.find_element(By.ID, e_id)

def find_className(cn, browser):
    return browser.find_element(By.CLASS_NAME, cn)
def finds_className(cn , browser):
    return browser.find_elements(By.CLASS_NAME, cn)

def find_linktext(lt, browser):
    return browser.find_element(By.LINK_TEXT, lt)

def find_name(name, browser):
    return browser.find_element(By.NAME, name)
def finds_name(name, browser):
    return browser.find_elements(By.NAME, name)

def find_tagName(tag_name, browser):
    return browser.find_element(By.TAG_NAME, tag_name)

def finds_tagName(tag_name, browser):
    return browser.find_elements(By.TAG_NAME, tag_name)

def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    base_path = getattr(sys, '_MEIPASS', os.path.dirname(os.path.abspath(__file__)))
    return os.path.join(base_path, relative_path)

In [2]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1080,800')
    options.add_argument('incognito')
    chrome_service = Service('chromedriver')
    chrome_service = Service(executable_path="chromedriver.exe")
    browser = webdriver.Chrome(service=chrome_service, options=options)
    
    return browser

In [10]:
def rm_login(rm_id, rm_pw, browser):
    browser.get("https://www.rm1.co.kr/")
    
    try:
        browser.switch_to.frame(0)
    except Exception as ex:
        print(ex)
        pass
    
    time.sleep(.5)
    find_id("stay_sign", browser).click()
    time.sleep(.5)
    try:
        alert = browser.switch_to.alert
        alert.accept()
    except Exception as ex:
        print(ex)
        pass
    
    find_id('user_id', browser).send_keys(rm_id)
    time.sleep(.5)
    find_id('user_pw', browser).send_keys(rm_pw)
    time.sleep(.5)
    find_id('user_pw', browser).send_keys("\n")
    
    time.sleep(2)
    
    try:
        find_id('closeToday', browser).click()
        time.sleep(.3)
        find_id('closeBtn', browser).click()
    except:
        pass

In [4]:
def first_search_company(business_number, browser):
    try:
        search_input = find_id('searchTxt', browser)
        search_input.clear()
        search_input.send_keys(business_number)
        time.sleep(1)
        search_input.send_keys('\n')

        time.sleep(1)
    except:
        browser.get("https://www.rm1.co.kr/")
        try:
            browser.switch_to.frame(0)
        except Exception as ex:
            print(ex)
            pass
        
        time.sleep(1)
        
        search_input = find_id('searchTxt', browser)
        search_input.clear()
        search_input.send_keys(business_number)
        time.sleep(1)
        search_input.send_keys('\n')

        time.sleep(1)

In [5]:
def extract_company_info(browser):
    # 첫번째 나온 회사 링크 클릭
    first_element = find_id('schResultCmpName', browser)
    find_tagName('a', first_element).click()

    time.sleep(3)
    
    soup = BS(browser.page_source, "html.parser")
    soup = soup.find(class_="tbl_l_a mt5")

    company_data = {}

    keys_to_extract = ['기업명', '대표자명', '사업자번호', '종업원수']

    for row in soup.select('table tbody tr'):
        th = row.find('th', scope='row')
        td = row.find('td').get_text(strip=True) if row.find('td') else None
        if th:
            key = th.get_text(strip=True)
            if key in keys_to_extract:
                company_data[key] = td

#     json_data = json.dumps(company_data, ensure_ascii=False, indent=4)
    
    return company_data

In [6]:
def extract_financial_statements(browser, company_data):
    # 재무상태표 click
    find_id('fsdtset1', browser).click()
    
    time.sleep(1)

    financial_statements = {}
    financial_statements_name = []

    soup = BS(browser.page_source, "html.parser")
    table = soup.find(id='fin_a')
    header = table.find('thead').find_all('th')[1:]

    for th in header:
        date = th.get_text(strip=True)
        financial_statements[date] = []

    rows = table.find('tbody').find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        for i, column in enumerate(columns):
            date = list(financial_statements.keys())[i]
            value = column.get_text(strip=True)
            financial_statements[date].append(value)
    
    try:
        financial_statements['2022-12-31(결산)'].insert(0, company_data['기업명'])
        financial_statements_name.insert(0, '회사이름')
    except:
        pass
    
    if '금액' in financial_statements:
        del financial_statements['금액']

    # print(json.dumps(financial_statements, ensure_ascii=False, indent=4))

    resoup = soup.find(class_="td_dp01")
    th_col_name = resoup.find_all('th')
    for col in th_col_name:
        financial_statements_name.append(col.text)

    return financial_statements_name, financial_statements

In [7]:
def extract_income_statement(browser, company_data):
    find_id('li12', browser).click()
    
    income_statement_value = {}
    income_statement_col_name = []

    soup = BS(browser.page_source, "html.parser")
    table = soup.find(id='fin_a')
    header = table.find('thead').find_all('th')[1:]

    for th in header:
        date = th.get_text(strip=True)
        income_statement_value[date] = []

    rows = table.find('tbody').find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        for i, column in enumerate(columns):
            date = list(income_statement_value.keys())[i]
            value = column.get_text(strip=True)
            income_statement_value[date].append(value)
    
    try:
        income_statement_value['2022-12-31(결산)'].insert(0, company_data['기업명'])
        income_statement_col_name.insert(0, '회사이름')
    except:
        pass
    
    if '금액' in income_statement_value:
        del income_statement_value['금액']

    # print(json.dumps(financial_statements, ensure_ascii=False, indent=4))

    resoup = soup.find(class_="td_dp01")
    th_col_name = resoup.find_all('th')
    for col in th_col_name:
        income_statement_col_name.append(col.text)
    
    return income_statement_col_name, income_statement_value

In [83]:
df = pd.read_excel('회사사업자번호.xlsx')
print(df.shape)
df['사업자번호'] = df['사업자번호'].astype('str')
print("중복치 확인 수 : ",df.duplicated().sum())
df.drop_duplicates(inplace=True)
print(df.shape)

(2152, 1)
중복치 확인 수 :  0
(2152, 1)


In [374]:
count = 1

final_company_data = []

final_financial_statements_data = []
final_income_statement_data = []

final_financial_statements_col_name = []
final_income_statement_col_name = []

browser = open_browser()

rm_login('rlaxodud3006', 'rlaxodud3006@', browser)

for business_number in df['사업자번호']:
    try:
        first_search_company(business_number, browser)

        time.sleep(1.5)
        try:
            company_data = extract_company_info(browser)
            final_company_data.append(company_data)

            financial_statements_name, financial_statements = extract_financial_statements(browser, company_data)

            income_statement_col_name, income_statement_value = extract_income_statement(browser, company_data)
        except Exception:
            pass

        try:
            if '2022-12-31(결산)' in income_statement_value:
                final_income_statement_data.append(income_statement_value['2022-12-31(결산)'])
                final_income_statement_col_name.append(income_statement_col_name)

            if '2022-12-31(결산)' in financial_statements:
                final_financial_statements_data.append(financial_statements['2022-12-31(결산)'])
                final_financial_statements_col_name.append(financial_statements_name)

        except Exception as e:
            print(f"An exception occurred: {str(e)}")
            pass

        print(count)
        count += 1
        
    except NoSuchElementException:
        print("Element not found. Logging in again.")
        rm_login('rlaxodud3006', 'rlaxodud3006@', browser)
        continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
Element not found. Logging in again.
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268

1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
Element not found. Logging in again.
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
Element not found. Logging in again.
1925
1926
1927
1

In [375]:
# 회사 이름 정보
print(len(final_company_data))

print(len(final_financial_statements_data))
print(len(final_income_statement_data))

print(len(final_financial_statements_col_name))
print(len(final_income_statement_col_name))

2236
1971
1971
1971
1971


In [362]:
df2 = pd.DataFrame(final_financial_statements_data)
df2_1 = pd.DataFrame(final_financial_statements_col_name)
df3 = pd.DataFrame(final_income_statement_data)
df3_1 = pd.DataFrame(final_income_statement_col_name)
df4 = pd.DataFrame(final_company_data)

In [363]:
df2.to_excel('재무제표1.xlsx', index=False)
df2_1.to_excel('재무제표1_col.xlsx', index=False)
df3.to_excel('손익계산서1.xlsx', index=False)
df3_1.to_excel('손익계산서1_col.xlsx', index=False)
df4.to_excel('회사인원정보1.xlsx', index=False)

In [36]:
df2 = pd.read_excel("C:\\Users\\yms06\\Project\\rm1_site_macro\\재무제표1.xlsx")
df2_1 = pd.read_excel("C:\\Users\\yms06\\Project\\rm1_site_macro\\재무제표1_col.xlsx")

list_of_lists = df2.values.tolist()
list_of_lists_col = df2_1.values.tolist()

final_list = []

for i, j in zip(list_of_lists_col, list_of_lists):
    final_list.append(i)
    final_list.append(j)
    
final_df = pd.DataFrame(final_list)

final_df.to_excel('재무제표 예시.xlsx', index=False)

In [129]:
df3 = pd.read_excel("C:\\Users\\yms06\\Project\\rm1_site_macro\\손익계산서1.xlsx")
df3_1 = pd.read_excel("C:\\Users\\yms06\\Project\\rm1_site_macro\\손익계산서1_col.xlsx")

list_of_lists1 = df3.values.tolist()
list_of_lists_col1 = df3_1.values.tolist()

final_list1 = []

for i, j in zip(list_of_lists_col1, list_of_lists1):
    final_list1.append(i)
    final_list1.append(j)
    
final_df1 = pd.DataFrame(final_list1)

final_df1.to_excel('손익계산서 예시.xlsx', index=False)

In [ ]:
browser = open_browser()

In [81]:
rm_login('rlaxodud3006', 'rlaxodud3006@', browser)

In [84]:
first_search_company(df['사업자번호'][0], browser)

In [85]:
company_data = extract_company_info(browser)

In [102]:
final_financial_statements_data = []
final_financial_statements_data.append(income_statement_col_name)
final_financial_statements_data.append(income_statement_value['2022-12-31(결산)'])
final_financial_statements_data

[['회사이름',
  '감사의견',
  '매출액',
  '총매출액',
  '상품매출액',
  '제품매출액',
  '[기타수입]',
  '매출원가',
  '상품매출원가',
  '기초재고',
  '매입',
  '타계정으로대체',
  '기말재고',
  '제품매출원가',
  '당기제품제조원가',
  '[기타원가]',
  '매출총이익(손실)',
  '판매비와관리비',
  '인건비',
  '급료와임금',
  '퇴직급여충당금전입액',
  '복리후생비',
  '주식보상비용',
  '일반관리비',
  '여비교통비',
  '통신비',
  '수도광열비',
  '세금과공과',
  '임차료비용',
  '감가상각비',
  '수선비',
  '보험료',
  '소모품비',
  '차량유지비',
  '교육훈련비',
  '수수료비용',
  '기타관리비',
  '판매비',
  '접대비',
  '광고선전비',
  '견본비',
  '기타판매비와관리비',
  '경상연구개발비',
  '대손상각비',
  '기타',
  '영업이익(손실)',
  '영업외수익',
  '이자수익',
  '수수료수익',
  '외환차익',
  '외화환산이익',
  '기타영업외수익',
  '영업외비용',
  '이자비용',
  '기부금',
  '외환차손',
  '외화환산손실',
  '유,무형,리스자산처분손실',
  '기타영업외비용',
  '법인세비용차감전계속사업이익(손실)',
  '계속사업손익법인세비용(부의법인세비용)',
  '계속사업이익(손실)',
  '중단사업이익(손실)',
  '[중단사업매출액]',
  '[중단사업매출원가]',
  '[중단사업매출총이익(손실)]',
  '[중단사업판매비와관리비]',
  '[중단사업영업이익(손실)]',
  '[중단사업영업외수익]',
  '[중단사업법인세비용차감전순이익(손실)]',
  '[중단사업손익법인세효과]',
  '당기순이익(손실)',
  '[포괄손익]',
  '기본주당계속사업손익',
  '기본주당순손익'],
 ['한국엠에스디(주)',
  '적정',
  '820,411,947',
  '820,41

In [103]:
result = dict(zip(final_financial_statements_data[0], final_financial_statements_data[1]))
result

{'회사이름': '한국엠에스디(주)',
 '감사의견': '적정',
 '매출액': '820,411,947',
 '총매출액': '820,411,947',
 '상품매출액': '806,555,140',
 '제품매출액': '13,856,807',
 '[기타수입]': '13,856,807',
 '매출원가': '680,309,151',
 '상품매출원가': '667,583,193',
 '기초재고': '164,322,142',
 '매입': '715,727,966',
 '타계정으로대체': '380,621',
 '기말재고': '212,086,294',
 '제품매출원가': '12,725,958',
 '당기제품제조원가': '12,725,958',
 '[기타원가]': '12,725,958',
 '매출총이익(손실)': '140,102,796',
 '판매비와관리비': '111,514,286',
 '인건비': '60,329,521',
 '급료와임금': '40,594,736',
 '퇴직급여충당금전입액': '8,241,900',
 '복리후생비': '10,374,236',
 '주식보상비용': '1,118,649',
 '일반관리비': '21,011,665',
 '여비교통비': '3,723,747',
 '통신비': '422,937',
 '수도광열비': '31,583',
 '세금과공과': '600,781',
 '임차료비용': '3,875,169',
 '감가상각비': '968,909',
 '수선비': '45,287',
 '보험료': '233,870',
 '소모품비': '592,376',
 '차량유지비': '15,222',
 '교육훈련비': '198,080',
 '수수료비용': '8,869,832',
 '기타관리비': '1,433,873',
 '판매비': '16,263,063',
 '접대비': '72,898',
 '광고선전비': '16,183,211',
 '견본비': '6,953',
 '기타판매비와관리비': '13,910,037',
 '경상연구개발비': '13,934,771',
 '대손상각비': '-24

In [105]:
selected_items = [
    '매출액', '매출원가', '판매비와관리비', '급료와임금', '퇴직급여충당금전입액',
    '복리후생비', '세금과공과', '임차료비용', '감가상각비', '경상연구개발비',
    '대손상각비', '영업이익(손실)', '이자수익', '이자비용'
]

filtered_data = {key: result[key] for key in selected_items}
print(filtered_data)

{'매출액': '820,411,947', '매출원가': '680,309,151', '판매비와관리비': '111,514,286', '급료와임금': '40,594,736', '퇴직급여충당금전입액': '8,241,900', '복리후생비': '10,374,236', '세금과공과': '600,781', '임차료비용': '3,875,169', '감가상각비': '968,909', '경상연구개발비': '13,934,771', '대손상각비': '-24,734', '영업이익(손실)': '28,588,510', '이자수익': '899,128', '이자비용': '222,517'}


In [106]:
re_df = pd.DataFrame(list(filtered_data.items()), columns=['항목', '값'])
re_df

,항목,값
0,매출액,"820,411,947"
1,매출원가,"680,309,151"
2,판매비와관리비,"111,514,286"
3,급료와임금,"40,594,736"
4,퇴직급여충당금전입액,"8,241,900"
5,복리후생비,"10,374,236"
6,세금과공과,"600,781"
7,임차료비용,"3,875,169"
8,감가상각비,"968,909"
9,경상연구개발비,"13,934,771"


In [108]:
print(len(list_of_lists1))
print(len(list_of_lists_col1))

1971
1971


In [116]:
# 손익계산서 전처리
final_dict = []

for i ,j in zip(list_of_lists_col1, list_of_lists1):
    result1 = dict(zip(i, j))
    final_dict.append(result1)
    
    
# 재무제표 전처리
final_dict1 = []

for i ,j in zip(list_of_lists_col, list_of_lists):
    result2 = dict(zip(i, j))
    final_dict1.append(result2)

In [121]:
final_lists = []

selected_items = [
    '매출액', '매출원가', '판매비와관리비', '급료와임금', '퇴직급여충당금전입액',
    '복리후생비', '세금과공과', '임차료비용', '감가상각비', '경상연구개발비',
    '대손상각비', '영업이익(손실)', '이자수익', '이자비용'
]

for ai in final_dict:
    try:
        filtered_data = {key: ai[key] for key in selected_items}
        final_lists.append(filtered_data)
    except:
        pass
    
print(len(final_lists))

388


In [136]:
column_names = ['유형자산(계)', '토지', '건물', '구축물', '(구축물감가상각누계액)', '기계장치', '(기계장치감가상각누계액)', '차량운반구', '(차량운반구감가상각누계액)', '공구와기구', '(공구와기구감가상각누계액)', '비품', '(비품감가상각누계액)', '기타유형자산', '(기타유형자산감가상각누계액)', '건설중인자산', '무형자산(계)', '(상각누계액)']
result_dict = {}

for company_data in final_dict1:
    company_name = company_data['회사이름']
    
    company_result = {column_name: 0 for column_name in column_names}
    
    for column_name in column_names:
        if column_name in company_data:
            value = company_data[column_name]
        else:
            value = 0
        
        company_result[column_name] = value
    
    result_dict[company_name] = company_result
    
print(len(result_dict))
print(result_dict)

1934
{'(주)서흥': {'유형자산(계)': '174,800,135', '토지': '30,819,112', '건물': '87,174,004', '구축물': 0, '(구축물감가상각누계액)': 0, '기계장치': '39,499,281', '(기계장치감가상각누계액)': 0, '차량운반구': 0, '(차량운반구감가상각누계액)': 0, '공구와기구': 0, '(공구와기구감가상각누계액)': 0, '비품': 0, '(비품감가상각누계액)': 0, '기타유형자산': '6,287,061', '(기타유형자산감가상각누계액)': 0, '건설중인자산': '10,444,191', '무형자산(계)': '4,752,005', '(상각누계액)': 0}, '(주)에프앤에프': {'유형자산(계)': '100,193,572', '토지': '27,097,311', '건물': '18,371,781', '구축물': '14,048,084', '(구축물감가상각누계액)': '5,872,257', '기계장치': 0, '(기계장치감가상각누계액)': 0, '차량운반구': '377,770', '(차량운반구감가상각누계액)': '182,064', '공구와기구': '7,799,193', '(공구와기구감가상각누계액)': '2,493,522', '비품': 0, '(비품감가상각누계액)': 0, '기타유형자산': 0, '(기타유형자산감가상각누계액)': 0, '건설중인자산': '12,763,562', '무형자산(계)': '12,062,481', '(상각누계액)': '2,199,191'}, '(주)케이엠': {'유형자산(계)': '58,101,582', '토지': '16,393,072', '건물': '29,389,255', '구축물': '310,975', '(구축물감가상각누계액)': '-', '기계장치': '7,759,883', '(기계장치감가상각누계액)': '-', '차량운반구': '795,239', '(차량운반구감가상각누계액)': '-', '공구와기구': '1,183,698', '(공구와기구감가상각누계액)': '-', '비

In [140]:
income_df = pd.DataFrame(result_dict).T
income_df.shape

(1934, 18)

In [145]:
income_df.to_excel("재무제표_final_예시.xlsx")